In [1]:
import random

# ============================================================
#                CARD / SHOE UTILITIES
# ============================================================

RANKS = list(range(1, 14))  # 1 = Ace, 11/12/13 = J/Q/K
UPCARDS = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]  # dealer upcard values (11 = Ace)


class Shoe:
    """
    Multi-deck shoe with a cut card.
    When the position reaches len(cards) - cut_cards, we reshuffle.

    Now also tracks Hi-Lo running count:
      +1 for 2–6
      -1 for 10–A
      0 for 7–9
    """

    def __init__(self, num_decks=6, cut_cards=60):
        self.num_decks = num_decks
        self.cut_cards = cut_cards
        self.cards = []
        self.position = 0
        self.running_count = 0  # Hi-Lo running count
        self._reshuffle()

    def _reshuffle(self):
        self.cards = RANKS * 4 * self.num_decks
        random.shuffle(self.cards)
        self.position = 0
        self.running_count = 0  # reset count at shuffle

    def draw(self):
        if self.position >= len(self.cards) - self.cut_cards:
            self._reshuffle()
        card = self.cards[self.position]
        self.position += 1

        # Update Hi-Lo running count
        if 2 <= card <= 6:
            self.running_count += 1
        elif card == 1 or card >= 10:
            self.running_count -= 1
        # 7,8,9 -> 0

        return card



def hand_value(hand):
    """
    Returns (total, soft):
      total = best total <= 21 if possible, otherwise minimal total
      soft  = True if at least one Ace is counted as 11 in the total
    """
    total = 0
    aces = 0
    for rank in hand:
        if rank == 1:        # Ace
            aces += 1
            total += 11
        elif rank >= 10:     # 10, J, Q, K
            total += 10
        else:
            total += rank

    while total > 21 and aces > 0:
        total -= 10
        aces -= 1

    soft = aces > 0  # at least one Ace still counted as 11
    return total, soft


def is_blackjack(hand):
    total, _ = hand_value(hand)
    return len(hand) == 2 and total == 21


def dealer_card_value(rank):
    """Convert rank (1..13) into dealer upcard value for strategy decisions."""
    if rank == 1:
        return 11  # Ace
    elif rank >= 10:
        return 10
    else:
        return rank


def pair_rank_numeric(rank):
    """
    Map a card rank (1..13) to a pair "rank" value:
      2..9 -> 2..9
      10/J/Q/K -> 10
      Ace -> 11
    Used for pair strategy tables.
    """
    if rank == 1:
        return 11
    elif rank >= 10:
        return 10
    else:
        return rank


# ============================================================
#                STRATEGY SECTION (EDIT HERE)
# ============================================================

# Strategy tables:
#   HARD_STRATEGY[(total, dealer_upcard)] = 'H' / 'S' / 'D'
#   SOFT_STRATEGY[(total, dealer_upcard)] = 'H' / 'S' / 'D'
#   PAIR_STRATEGY[(pair_rank, dealer_upcard)] = 'P' / 'H' / 'S' / 'D'
#
# Legend:
#   H = Hit
#   S = Stand
#   D = Double (if not allowed, treated as Hit)
#   P = Split
#
# These are initialized to a reasonable EV-optimal style
# basic strategy for 6D, H17, DAS. You can freely edit them.

HARD_STRATEGY = {}
SOFT_STRATEGY = {}
PAIR_STRATEGY = {}


def generate_default_strategy_tables():
    """
    Fill HARD_STRATEGY, SOFT_STRATEGY, PAIR_STRATEGY
    using a basic-strategy approximation (EV-based) for:

      - 6 decks
      - Dealer hits soft 17 (H17)
      - Double after split allowed
      - Only one card after split aces (handled in play logic)

    You can modify this function OR adjust the dictionaries
    AFTER calling it if you want custom strategies.
    """

    # --------- HARD TOTALS ---------
    # Your rules:
    #   Dealer 7–A: hit until 17+ (stand on 17+)
    #   Dealer 4–6: stand on 12+
    #   Dealer 2–3: stand on 13+
    #   Doubling:
    #       total 11: always double
    #       total 10: double vs 2–9 (not 10 or A)
    #       total  9: double vs 2–6

    for up in UPCARDS:  # dealer upcard
        for total in range(4, 22):  # possible hard totals
            # Base H/S rule
            if up in (7, 8, 9, 10, 11):  # dealer good upcard
                action = 'S' if total >= 17 else 'H'
            elif up in (4, 5, 6):  # dealer poor upcard
                action = 'S' if total >= 12 else 'H'
            else:  # up in (2, 3) dealer fair upcard
                action = 'S' if total >= 13 else 'H'

            # Doubling overrides for hard totals
            if total == 11:
                action = 'D'
            elif total == 10:
                if up not in (10, 11):
                    action = 'D'
            elif total == 9:
                if 2 <= up <= 6:
                    action = 'D'

            HARD_STRATEGY[(total, up)] = action

    # --------- SOFT TOTALS ---------
    # Base rule from your text: with a soft hand, hit until at least 18.
    for up in UPCARDS:
        for total in range(12, 22):  # soft totals A+Ace..A+10
            if total >= 18:
                action = 'S'
            else:
                action = 'H'
            SOFT_STRATEGY[(total, up)] = action

    # Now overlay EV-based soft doubles (standard chart style for 6D H17, DAS):
    # Let soft totals be: A+2=13, A+3=14, A+4=15, A+5=16, A+6=17, A+7=18, A+8=19, A+9=20
    # Rules:
    #   A,2 (13) / A,3 (14): double vs 5–6
    #   A,4 (15) / A,5 (16): double vs 4–6
    #   A,6 (17):            double vs 3–6
    #   A,7 (18):            double vs 3–6; stand vs 2,7,8; hit vs 9,10,A
    #   A,8 (19):            double vs 6 (otherwise stand)
    #   A,9 (20):            stand

    # A,2 & A,3
    for total in (13, 14):
        for up in (5, 6):
            SOFT_STRATEGY[(total, up)] = 'D'
    # A,4 & A,5
    for total in (15, 16):
        for up in (4, 5, 6):
            SOFT_STRATEGY[(total, up)] = 'D'
    # A,6
    for up in (3, 4, 5, 6):
        SOFT_STRATEGY[(17, up)] = 'D'
    # A,7
    for up in (3, 4, 5, 6):
        SOFT_STRATEGY[(18, up)] = 'D'
    for up in (2, 7, 8):
        SOFT_STRATEGY[(18, up)] = 'S'
    for up in (9, 10, 11):
        SOFT_STRATEGY[(18, up)] = 'H'
    # A,8
    for up in UPCARDS:
        SOFT_STRATEGY[(19, up)] = 'S'
    SOFT_STRATEGY[(19, 6)] = 'D'
    # A,9: always stand
    for up in UPCARDS:
        SOFT_STRATEGY[(20, up)] = 'S'

    # --------- PAIR STRATEGY ---------
    # Rules:
    #   Always split Aces and 8s
    #   Never split 10s, 5s, or 4s
    #   Split 2s, 3s, 7s unless dealer has 8, 9, 10, or Ace
    #   Split 6s vs dealer 2–6
    #
    # When not splitting, treat it as the corresponding hard total.

    for up in UPCARDS:
        for pr in range(2, 12):  # pair rank: 2..10, 11=Ace
            split = False

            if pr == 11 or pr == 8:
                # Always split Aces and 8s
                split = True
            elif pr in (10, 5, 4):
                split = False
            elif pr in (2, 3, 7):
                if up not in (8, 9, 10, 11):
                    split = True
            elif pr == 6:
                if 2 <= up <= 6:
                    split = True

            if split:
                action = 'P'
            else:
                # Fallback: use hard-total action for total = pair_total
                total = pr * 2
                if total < 4:
                    total = 4
                if total > 21:
                    total = 21
                action = HARD_STRATEGY.get((total, up), 'H')
            PAIR_STRATEGY[(pr, up)] = action


def basic_strategy_decision_from_tables(hand, dealer_upcard, can_split, can_double):
    """
    Decide action using the strategy tables above.
    Returns: "HIT", "STAND", "DOUBLE", or "SPLIT"
    """
    total, soft = hand_value(hand)
    d_val = dealer_card_value(dealer_upcard)

    # --- Check for split opportunities ---
    if can_split and len(hand) == 2 and hand[0] == hand[1]:
        pr = pair_rank_numeric(hand[0])  # 2..10, 11 = Ace
        action = PAIR_STRATEGY.get((pr, d_val))
        if action == 'P':
            return "SPLIT"
        # If not splitting, fall through to hard/soft logic

    # --- Use hard/soft tables ---
    if soft:
        action = SOFT_STRATEGY.get((total, d_val))
        # Safety fallback: soft rule "hit until 18"
        if action is None:
            action = 'S' if total >= 18 else 'H'
    else:
        action = HARD_STRATEGY.get((total, d_val))
        # Safety fallback: use rough hard-hand logic if missing
        if action is None:
            if d_val in (7, 8, 9, 10, 11):
                action = 'S' if total >= 17 else 'H'
            elif d_val in (4, 5, 6):
                action = 'S' if total >= 12 else 'H'
            else:
                action = 'S' if total >= 13 else 'H'

    # Interpret action char
    if action == 'S':
        return "STAND"
    elif action == 'H':
        return "HIT"
    elif action == 'D':
        # Double if allowed, otherwise hit
        return "DOUBLE" if can_double else "HIT"
    else:
        # Unknown char; default to hit
        return "HIT"


def print_strategy_tables():
    """Print hard, soft, and pair strategy tables nicely."""

    # ----- Hard table -----
    print("=== HARD TOTALS STRATEGY (H/S/D) ===")
    header = "Tot | " + " ".join(f"{u:>2}" if u != 11 else " A" for u in UPCARDS)
    print(header)
    print("-" * len(header))
    for total in range(4, 22):
        row = f"{total:>3} | "
        for up in UPCARDS:
            a = HARD_STRATEGY.get((total, up), ' ')
            row += f"{a:>2} "
        print(row)

    # ----- Soft table -----
    print("\n=== SOFT TOTALS STRATEGY (A+X, H/S/D) ===")
    header = "Tot | " + " ".join(f"{u:>2}" if u != 11 else " A" for u in UPCARDS)
    print(header)
    print("-" * len(header))
    for total in range(12, 22):
        row = f"{total:>3} | "
        for up in UPCARDS:
            a = SOFT_STRATEGY.get((total, up), ' ')
            row += f"{a:>2} "
        print(row)

    # ----- Pair table -----
    print("\n=== PAIR STRATEGY (P/H/S/D) ===")
    header = "Pr  | " + " ".join(f"{u:>2}" if u != 11 else " A" for u in UPCARDS)
    print(header)
    print("-" * len(header))
    for pr in range(2, 12):
        label = str(pr) if pr <= 10 else "A"
        row = f"{label:>3} | "
        for up in UPCARDS:
            a = PAIR_STRATEGY.get((pr, up), ' ')
            row += f"{a:>2} "
        print(row)
    print()



# ============================================================
#                PLAYER & DEALER PLAY
# ============================================================

def play_hand(hand, dealer_upcard, shoe, bet=1.0,
              max_splits=3, split_depth=0, is_split_aces=False):
    """
    Plays a single player starting hand according to strategy tables.
    Handles splits recursively.

    "One card only after split Aces" is enforced by is_split_aces flag:
      - after deal to a split ace, we immediately stand that hand.
    Returns a list of dicts:
      {'cards': [...], 'bet': bet_amount, 'busted': bool}
    """

    results = []

    def can_split_func(h, depth, split_aces_flag):
        # No further splitting after split aces or max depth
        if split_aces_flag:
            return False
        if depth >= max_splits:
            return False
        return len(h) == 2 and h[0] == h[1]

    while True:
        total, soft = hand_value(hand)

        # Bust
        if total > 21:
            results.append({'cards': hand, 'bet': bet, 'busted': True})
            return results

        # Split aces: only one card to each ace, then stand
        if is_split_aces and len(hand) >= 2:
            results.append({'cards': hand, 'bet': bet, 'busted': False})
            return results

        can_split_now = can_split_func(hand, split_depth, is_split_aces)
        can_double_now = (len(hand) == 2)

        action = basic_strategy_decision_from_tables(
            hand, dealer_upcard, can_split_now, can_double_now
        )

        if action == "STAND":
            results.append({'cards': hand, 'bet': bet, 'busted': False})
            return results

        elif action == "HIT":
            hand = hand + [shoe.draw()]
            continue

        elif action == "DOUBLE" and can_double_now:
            bet *= 2
            hand = hand + [shoe.draw()]
            total, _ = hand_value(hand)
            busted = total > 21
            results.append({'cards': hand, 'bet': bet, 'busted': busted})
            return results

        elif action == "SPLIT" and can_split_now:
            rank = hand[0]
            hand1 = [rank, shoe.draw()]
            hand2 = [rank, shoe.draw()]
            split_aces_next = (rank == 1)
            res1 = play_hand(hand1, dealer_upcard, shoe,
                             bet=bet,
                             max_splits=max_splits,
                             split_depth=split_depth + 1,
                             is_split_aces=split_aces_next)
            res2 = play_hand(hand2, dealer_upcard, shoe,
                             bet=bet,
                             max_splits=max_splits,
                             split_depth=split_depth + 1,
                             is_split_aces=split_aces_next)
            return res1 + res2

        else:
            # Fallback: treat unknown action as hit
            hand = hand + [shoe.draw()]
            continue


def dealer_play(hand, shoe):
    """
    Dealer plays according to:
      - Dealer HITS on soft 17 (H17)
      - Stands on hard 17+ and soft 18+
    Returns (total, busted_bool)
    """
    while True:
        total, soft = hand_value(hand)
        # Stand on: hard 17+, or soft 18+
        if total > 17:
            return total, total > 21
        if total == 17 and not soft:
            return total, False  # hard 17 stands
        # Otherwise hit (including soft 17)
        hand.append(shoe.draw())


# ============================================================
#                ROUND & SIMULATION
# ============================================================

def play_round(shoe, bet=1.0):
    """
    Plays one full round: 1 player vs dealer.
    Returns profit from player's point of view, in units of the initial bet.
    """

    # Initial deal
    player = [shoe.draw(), shoe.draw()]
    dealer = [shoe.draw(), shoe.draw()]  # dealer[0] is upcard

    # Check naturals
    player_bj = is_blackjack(player)
    dealer_bj = is_blackjack(dealer)

    if player_bj or dealer_bj:
        if player_bj and not dealer_bj:
            return 1.5 * bet  # Blackjack 3:2
        elif dealer_bj and not player_bj:
            return -bet
        else:
            return 0.0  # push

    # No naturals: player plays
    dealer_upcard = dealer[0]
    player_results = play_hand(player, dealer_upcard, shoe, bet=bet)

    # If all hands busted, dealer does not play
    if all(h['busted'] for h in player_results):
        profit = 0.0
        for h in player_results:
            profit -= h['bet']
        return profit

    # Dealer plays
    dealer_total, dealer_busted = dealer_play(dealer, shoe)

    profit = 0.0
    for h in player_results:
        if h['busted']:
            profit -= h['bet']
        else:
            player_total, _ = hand_value(h['cards'])
            if dealer_busted:
                profit += h['bet']
            else:
                if player_total > dealer_total:
                    profit += h['bet']
                elif player_total < dealer_total:
                    profit -= h['bet']
                # equal => push
    return profit

# ============================================================
#      SIDE BET HELPERS: 6-DECK FULL CARDS (RANK + SUIT)
# ============================================================

def build_6deck_side_deck():
    """
    Build a full 6-deck shoe with explicit ranks and suits
    for side-bet evaluation (Low3, Top3, Lucky7).

    Card format: (rank, suit)
      rank: 1..13 (1 = Ace, 11/12/13 = J/Q/K)
      suit: 'C', 'D', 'H', 'S'
    """
    suits = ['C', 'D', 'H', 'S']  # Clubs, Diamonds, Hearts, Spades
    deck = []
    for _ in range(6):
        for r in range(1, 14):
            for s in suits:
                deck.append((r, s))
    return deck


def card_ranks(cards):
    return [c[0] for c in cards]


def card_suits(cards):
    return [c[1] for c in cards]


def is_flush(cards):
    s = card_suits(cards)
    return s[0] == s[1] == s[2]


def is_three_of_a_kind(cards):
    r = card_ranks(cards)
    return r[0] == r[1] == r[2]


def is_straight(cards):
    """
    3-card straight:
      - A can be low (A-2-3) or high (Q-K-A)
      - No wrap-around like K-A-2
    """
    r = sorted(card_ranks(cards))
    u = sorted(set(r))
    if len(u) != 3:
        return False

    # Case 1: A is low (treat Ace as 1)
    a, b, c = u
    if b == a + 1 and c == b + 1:
        return True

    # Case 2: A is high (treat Ace as 14)
    u2 = [14 if x == 1 else x for x in u]
    u2.sort()
    a, b, c = u2
    return b == a + 1 and c == b + 1


def is_straight_flush(cards):
    return is_flush(cards) and is_straight(cards)


def is_suited_three_of_a_kind(cards):
    return is_three_of_a_kind(cards) and is_flush(cards)


def card_color(suit):
    """Return 'R' for red, 'B' for black."""
    return 'R' if suit in ('D', 'H') else 'B'

# ============================================================
#                SIDE BET SIMULATION
# ============================================================

def simulate_side_bets(num_trials=500000, jackpot=0.0, seed=None):
    """
    Simulate side bets using a fresh 6-deck pack each trial
    (no interaction with main blackjack shoe).

    Side bets:
      - Low3:
          * Based on your 2 cards + dealer upcard (3 cards total)
          * Pays 9:1 for any Straight, Flush, or Three of a Kind
            (including Straight Flush or Suited Trips)
      - Top3:
          * Also on the same 3 cards
          * Pays:
              - 90:1 for any Three of a Kind
              - 180:1 for Straight Flush
              - 270:1 for Suited Three of a Kind
          * You must bet Low3 to be allowed to bet Top3, but EV per unit
            of Top3 is independent, so we compute it separately.
      - Lucky7:
          * Based on first two player cards and dealer upcard:
              - 2:1   if exactly one of your two cards is a 7
              - 25:1  if both of your cards are 7, dealer upcard not 7
              - 200:1 if both of your cards and dealer upcard are 7
              - +10%  of jackpot if those three 7s are same color (red/black)
              - +100% of jackpot if those three 7s are same suit
            Only the highest category applies. For 3 suited 7s, we interpret
            it as 200:1 + 100% of jackpot (not 10% + 100%).

    Returns a dict with:
      - 'low3_edge'
      - 'top3_edge'
      - 'lucky7_edge_for_jackpot'
      - 'lucky7_break_even_jackpot'
    """
    if seed is not None:
        random.seed(seed)

    deck = build_6deck_side_deck()
    N = num_trials

    low3_profit = 0.0
    top3_profit = 0.0
    lucky7_profit_base = 0.0  # EV with jackpot = 0
    jackpot_weight_sum = 0.0  # sum of 0 / 0.1 / 1 across trials

    for _ in range(N):
        # Sample 3 distinct cards from full 6-deck pack
        c1, c2, c3 = random.sample(deck, 3)
        player1, player2, dealer = c1, c2, c3
        three_cards = [player1, player2, dealer]

        # ---------- Low3 ----------
        flush = is_flush(three_cards)
        trips = is_three_of_a_kind(three_cards)
        straight = is_straight(three_cards)

        low3_win = straight or flush or trips
        if low3_win:
            low3_profit += 9.0    # net profit 9:1
        else:
            low3_profit -= 1.0    # lose the bet

        # ---------- Top3 ----------
        straight_flush = is_straight_flush(three_cards)
        suited_trips = is_suited_three_of_a_kind(three_cards)

        if suited_trips:
            # Highest Top3 category
            top3_profit += 270.0
        elif straight_flush:
            top3_profit += 180.0
        elif trips:
            top3_profit += 90.0
        else:
            top3_profit -= 1.0

        # ---------- Lucky7 ----------
        r1, s1 = player1
        r2, s2 = player2
        r3, s3 = dealer

        ranks_player = [r1, r2]
        ranks_total = [r1, r2, r3]

        count7_player = ranks_player.count(7)
        count7_total = ranks_total.count(7)

        profit = -1.0
        jackpot_weight = 0.0  # 0, 0.1, or 1 depending on special 7-7-7

        if count7_player == 1 and count7_total == 1:
            # Exactly one of your first two cards is a 7
            profit = 2.0
        elif count7_player == 2 and count7_total == 2:
            # Both of your first two cards are 7, dealer not 7
            profit = 25.0
        elif count7_player == 2 and count7_total == 3:
            # Both of your cards and dealer upcard are 7
            suits = [s1, s2, s3]
            colors = [card_color(s) for s in suits]
            same_suit = len(set(suits)) == 1
            same_color = len(set(colors)) == 1

            if same_suit:
                # Suited 7-7-7: 200:1 + 100% of jackpot
                profit = 200.0
                jackpot_weight = 1.0
            elif same_color:
                # Same-color (but not suited) 7-7-7: 200:1 + 10% of jackpot
                profit = 200.0
                jackpot_weight = 0.1
            else:
                # Plain 7-7-7: 200:1, no jackpot
                profit = 200.0
                jackpot_weight = 0.0

        lucky7_profit_base += profit
        jackpot_weight_sum += jackpot_weight

    # Convert to edges (expected net profit per 1-unit bet)
    low3_edge = low3_profit / N
    top3_edge = top3_profit / N

    # Lucky7: EV(J) = EV_base + J * (jackpot_weight_sum / N)
    EV_base = lucky7_profit_base / N
    jackpot_coeff = jackpot_weight_sum / N  # average jackpot weight per hand

    if jackpot_coeff > 0:
        # J* such that EV(J*) = 0
        lucky7_break_even_jackpot = -EV_base / jackpot_coeff
    else:
        lucky7_break_even_jackpot = float('inf')

    lucky7_edge_for_jackpot = EV_base + jackpot * jackpot_coeff

    return {
        'low3_edge': low3_edge,
        'top3_edge': top3_edge,
        'lucky7_edge_for_jackpot': lucky7_edge_for_jackpot,
        'lucky7_break_even_jackpot': lucky7_break_even_jackpot,
        'lucky7_base_edge_no_jackpot': EV_base,
    }

# ============================================================
#    LUCKY 7 EDGE GIVEN n SEVENS IN 3 DECKS (WITH CUT CARD)
# ============================================================

# ============================================================
#   LUCKY 7 EDGE: 3 DECKS, CUT AT 60, EXACTLY n7 SEVENS
# ============================================================

def simulate_lucky7_three_decks_with_cut(n7, num_trials=100000, jackpot=0.0, seed=None):
    """
    Simulate the Lucky 7 side bet in a 3-deck slug with a cut card:

      - Total cards in slug: 3 decks = 156 cards.
      - Cut card: when 60 cards remain in the shoe, no more betting.
      - Exactly n7 of those 156 cards are 7s, spread randomly throughout.
      - Remaining (156 - n7) cards are non-7s (never trigger Lucky 7).

    For each trial:
      - Build the 156-card deck, mark n7 of them as 7s (with random suits),
        shuffle, then deal through the shoe:
            * While cards_remaining > 60 and >= 3:
                - One Lucky 7 bet is placed.
                - Deal 2 player cards + 1 dealer upcard.
                - Evaluate Lucky 7 payout for that bet.
      - Track total profit and jackpot-weight across all bets.

    Lucky 7 payouts per 1-unit bet:
      - 2:1   if exactly one of your first two cards is a 7
      - 25:1  if both of your first two cards are 7 and dealer upcard is not 7
      - 200:1 if both of your first two cards and dealer upcard are 7
      - +10%  of jackpot if those three 7s are same color (red/black)
      - +100% of jackpot if those three 7s are same suit

      For 3 suited 7s, we treat it as:
        200:1 + 100% of jackpot (not 10% + 100%).

    Returns a dict with:
      - 'lucky7_edge_for_jackpot': EV with the given jackpot (per unit bet)
      - 'lucky7_break_even_jackpot': J* s.t. EV(J*) = 0
      - 'lucky7_base_edge_no_jackpot': EV when jackpot = 0
      - 'jackpot_weight_per_bet': average jackpot multiplier per bet
      - 'bets_per_shoe': average number of Lucky 7 bets before the cut
    """
    if seed is not None:
        random.seed(seed)

    TOTAL_CARDS = 3 * 52  # 156
    CUT_CARDS = 60        # stop betting once 60 remain

    if not (0 <= n7 <= 24):
        raise ValueError("With 3 decks, n7 must be between 0 and 24 inclusive.")

    total_profit_base = 0.0
    total_jackpot_weight = 0.0
    total_bets = 0
    total_bets_per_shoe = 0

    suits_all = ['C', 'D', 'H', 'S']

    for _ in range(num_trials):
        # Build deck: n7 sevens with random suits, rest non-7s (suit irrelevant)
        deck = []
        for _ in range(n7):
            deck.append(('7', random.choice(suits_all)))
        for _ in range(TOTAL_CARDS - n7):
            deck.append(('X', None))  # non-7

        random.shuffle(deck)

        position = 0
        bets_this_shoe = 0

        while True:
            cards_remaining = TOTAL_CARDS - position
            # Stop if we're at or past the cut, or can't deal a full hand
            if cards_remaining <= CUT_CARDS or cards_remaining < 3:
                break

            # One Lucky 7 side bet
            bets_this_shoe += 1
            total_bets += 1

            # Deal player1, player2, dealer
            c1 = deck[position]
            c2 = deck[position + 1]
            c3 = deck[position + 2]
            position += 3

            is7_p1 = (c1[0] == '7')
            is7_p2 = (c2[0] == '7')
            is7_d  = (c3[0] == '7')
            s1, s2, s3 = c1[1], c2[1], c3[1]

            count7_player = int(is7_p1) + int(is7_p2)
            count7_total  = count7_player + int(is7_d)

            profit = -1.0
            jackpot_weight = 0.0

            if count7_player == 1 and count7_total == 1:
                # Exactly one of your first two cards is a 7
                profit = 2.0
            elif count7_player == 2 and count7_total == 2:
                # Both of your first two cards are 7, dealer not 7
                profit = 25.0
            elif count7_player == 2 and count7_total == 3:
                # Both of your cards and dealer upcard are 7
                suits7 = [s1, s2, s3]
                colors7 = [card_color(s) for s in suits7]
                same_suit = len(set(suits7)) == 1
                same_color = len(set(colors7)) == 1

                if same_suit:
                    # 7-7-7 all same suit: 200:1 + 100% jackpot
                    profit = 200.0
                    jackpot_weight = 1.0
                elif same_color:
                    # 7-7-7 same color, not all same suit: 200:1 + 10% jackpot
                    profit = 200.0
                    jackpot_weight = 0.1
                else:
                    # 7-7-7 mixed colors: 200:1, no jackpot
                    profit = 200.0
                    jackpot_weight = 0.0

            total_profit_base += profit
            total_jackpot_weight += jackpot_weight

        total_bets_per_shoe += bets_this_shoe

    if total_bets == 0:
        # No bets happened (e.g., very small n7 and extreme settings) – guard
        return {
            'lucky7_edge_for_jackpot': 0.0,
            'lucky7_break_even_jackpot': float('inf'),
            'lucky7_base_edge_no_jackpot': 0.0,
            'jackpot_weight_per_bet': 0.0,
            'bets_per_shoe': 0.0,
        }

    EV_base = total_profit_base / total_bets
    jackpot_coeff = total_jackpot_weight / total_bets
    bets_per_shoe = total_bets_per_shoe / num_trials

    if jackpot_coeff > 0:
        lucky7_break_even_jackpot = -EV_base / jackpot_coeff
    else:
        lucky7_break_even_jackpot = float('inf')

    lucky7_edge_for_jackpot = EV_base + jackpot * jackpot_coeff

    return {
        'lucky7_edge_for_jackpot': lucky7_edge_for_jackpot,
        'lucky7_break_even_jackpot': lucky7_break_even_jackpot,
        'lucky7_base_edge_no_jackpot': EV_base,
        'jackpot_weight_per_bet': jackpot_coeff,
        'bets_per_shoe': bets_per_shoe,
    }


# ============================================================
#                SIMULATE
# ============================================================

def simulate(num_rounds=200000, num_decks=6, cut_cards=60, seed=None):
    """
    Monte Carlo estimation of player edge per initial bet.
    Returns estimated edge (expected profit per hand, in units of the initial bet).
    """
    if seed is not None:
        random.seed(seed)

    shoe = Shoe(num_decks=num_decks, cut_cards=cut_cards)
    total_profit = 0.0
    for _ in range(num_rounds):
        total_profit += play_round(shoe, bet=1.0)

    edge = total_profit / num_rounds
    return edge

def simulate_counter(num_rounds=200000, num_decks=6, cut_cards=60, seed=None):
    """
    Monte Carlo estimation of the *card counter's* edge with betting strategy:
      - Hi-Lo count:
          +1 for 2–6
          -1 for 10–A
          0 for 7–9
      - True count = running_count / decks_remaining
        (decks_remaining = unseen cards / 52)
      - Bet 1 unit when floor(true count) < 2
      - Bet 5 units when floor(true count) >= 2

    Returns:
      edge = total_profit / total_initial_bet
    """
    if seed is not None:
        random.seed(seed)

    shoe = Shoe(num_decks=num_decks, cut_cards=cut_cards)
    total_profit = 0.0
    total_initial_bet = 0.0

    for _ in range(num_rounds):
        # Compute true count before placing the bet
        cards_remaining = len(shoe.cards) - shoe.position
        decks_remaining = cards_remaining / 52.0 if cards_remaining > 0 else 1.0

        true_count = shoe.running_count / decks_remaining

        # Use floored true count (toward zero), like typical card counting
        if true_count >= 0:
            tc_int = int(true_count)
        else:
            tc_int = -int(abs(true_count))
            
        if tc_int >= 2: 
            bet = 5.0
            
        elif tc_int == 1:
            bet = 2.5
            
        else: 
            bet = 1.0

        total_initial_bet += bet
        profit = play_round(shoe, bet=bet)
        total_profit += profit

    edge = total_profit / total_initial_bet
    return edge


# ============================================================
#                MAIN
# ============================================================
if __name__ == "__main__":
    # Build the default tables with soft doubles allowed
    generate_default_strategy_tables()

    # Print strategy tables so you can inspect/edit them
    print_strategy_tables()

    # ---- Main flat-bet blackjack simulation ----
    rounds = 300000
    est_edge = simulate(num_rounds=rounds, num_decks=6, cut_cards=60, seed=42)
    print(f"Simulated hands (flat bet): {rounds}")
    print(f"Estimated player edge per initial bet (flat): {est_edge:.4%}")
    print("(Negative means the house has the edge.)")

    # ---- Card counter simulation ----
    counter_rounds = 300000
    counter_edge = simulate_counter(
        num_rounds=counter_rounds,
        num_decks=6,
        cut_cards=60,
        seed=99
    )
    print(f"\nSimulated hands (counter): {counter_rounds}")
    print("Betting 1 unit when TC < 2, 5 units when TC ≥ 2")
    print(f"Estimated card counter edge per unit bet: {counter_edge:.4%}")


    print("\n=== SIDE BET EDGES (per unit bet) ===")
    print(f"Low3 edge:  {side_results['low3_edge']:.4%}")
    print(f"Top3 edge:  {side_results['top3_edge']:.4%}")

    print(f"\nLucky7 base edge (no jackpot): {side_results['lucky7_base_edge_no_jackpot']:.4%}")
    print(
        f"Lucky7 edge with jackpot = {current_jackpot:.0f}: "
        f"{side_results['lucky7_edge_for_jackpot']:.4%}"
    )
    print(
        "Lucky7 break-even jackpot (edge = 0): "
        f"{side_results['lucky7_break_even_jackpot']:.2f} "
        "(in units of the side bet)"
    )
    
    n7 = 6              # say you know there are 6 sevens in this 3-deck segment
    current_jackpot = 5000.0  # in units of the side bet (e.g., $5000 for $1 bet)

    res = simulate_lucky7_three_decks_with_cut(
        n7=n7,
        num_trials=100000,
        jackpot=current_jackpot,
        seed=2025
    )

    print(f"\n=== Lucky 7 (3 decks, cut 60, n7={n7}) ===")
    print(f"Average Lucky 7 bets per shoe: {res['bets_per_shoe']:.2f}")
    print(f"Base edge (no jackpot): {res['lucky7_base_edge_no_jackpot']:.4%}")
    print(
        f"Edge with jackpot = {current_jackpot:.0f}: "
        f"{res['lucky7_edge_for_jackpot']:.4%}"
    )
    print(
        "Break-even jackpot (edge = 0): "
        f"{res['lucky7_break_even_jackpot']:.2f} "
        "(in units of the side bet)"
    )

=== HARD TOTALS STRATEGY (H/S/D) ===
Tot |  2  3  4  5  6  7  8  9 10  A
-----------------------------------
  4 |  H  H  H  H  H  H  H  H  H  H 
  5 |  H  H  H  H  H  H  H  H  H  H 
  6 |  H  H  H  H  H  H  H  H  H  H 
  7 |  H  H  H  H  H  H  H  H  H  H 
  8 |  H  H  H  H  H  H  H  H  H  H 
  9 |  D  D  D  D  D  H  H  H  H  H 
 10 |  D  D  D  D  D  D  D  D  H  H 
 11 |  D  D  D  D  D  D  D  D  D  D 
 12 |  H  H  S  S  S  H  H  H  H  H 
 13 |  S  S  S  S  S  H  H  H  H  H 
 14 |  S  S  S  S  S  H  H  H  H  H 
 15 |  S  S  S  S  S  H  H  H  H  H 
 16 |  S  S  S  S  S  H  H  H  H  H 
 17 |  S  S  S  S  S  S  S  S  S  S 
 18 |  S  S  S  S  S  S  S  S  S  S 
 19 |  S  S  S  S  S  S  S  S  S  S 
 20 |  S  S  S  S  S  S  S  S  S  S 
 21 |  S  S  S  S  S  S  S  S  S  S 

=== SOFT TOTALS STRATEGY (A+X, H/S/D) ===
Tot |  2  3  4  5  6  7  8  9 10  A
-----------------------------------
 12 |  H  H  H  H  H  H  H  H  H  H 
 13 |  H  H  H  D  D  H  H  H  H  H 
 14 |  H  H  H  D  D  H  H  H  H  H 

NameError: name 'side_results' is not defined